# CNN for Image Classification

### Import Libraries

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import random_split, ConcatDataset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torcheval.metrics.functional import (multiclass_accuracy, 
                                          multiclass_confusion_matrix, 
                                          multiclass_precision, 
                                          multiclass_recall)
from sklearn.metrics import ConfusionMatrixDisplay, recall_score, precision_score, accuracy_score

### Load the data

In [ ]:
# Batch size for training, validation and testing datasets
batch_size = 64

# Percentages for training, validation and training sets
train_split = 0.6
valid_split = 0.2
test_split = 0.2

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

In [ ]:
train_data = datasets.CIFAR10('./data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10('./data', train=False, transform=transform, download=True)

In [ ]:
# Concatenate the datasets
full_dataset = ConcatDataset([train_data, test_data])

len_full_dataset = len(full_dataset)
print("Full dataset length", len_full_dataset)

In [ ]:
# Split data into training, validation and test datasets

# Seed the generator to achieve the same splits everytime
split_generator = torch.Generator().manual_seed(42) 

train_size = int(np.floor(train_split * len_full_dataset))
valid_size = int(np.floor(valid_split * len_full_dataset))
test_size = int(np.floor(test_split * len_full_dataset))

train_dataset, valid_dataset, test_dataset = random_split(full_dataset, 
                                                               [train_size, valid_size, test_size], 
                                                               split_generator)

In [ ]:
print("Train dataset length: ", len(train_dataset))
print("Validation dataset length: ", len(valid_dataset))
print("Test dataset length: ", len(test_dataset))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)

In [ ]:
# Image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Visualize a batch from dataset

In [ ]:
def imshow(img):
    # Un-normalize the images
    img = img * 0.5 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0))) # Change the shape

In [ ]:
images, labels = next(iter(train_loader))
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
print("Batch Shape: ", images.shape)

### Define the Network Architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # First convolution block
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        
        # Second covolution block
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        
        # Third convolution block
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Linear layer (4 * 4 * 128 -> 128)
        self.fc1 = nn.Linear(4 * 4 * 128, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Dropout layer
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        self.dropout4 = nn.Dropout(0.5)
        
    def forward(self, x):
        
        # First block
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        
        # Second block
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout2(x)
        
        # Third block
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        x = self.dropout3(x)
        
        # Flattern
        x = x.view(-1, 4 * 4 * 128)
        
        # Linear layer
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)

        x = self.fc2(x)
        
        return x

In [ ]:
# create the CNN
model = Net()
model

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

In [ ]:
if train_on_gpu:
    model.cuda()

### Loss Function & Optimizer

In [ ]:
# Categorical cross entrophy loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

### Train the Network

In [ ]:
n_epochs = 20

valid_loss_min = np.Inf

train_losses, valid_losses = [], []

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    
    ############# Train the model #############
    model.train()
    for images, labels in train_loader:
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
            
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
    ############# Train the model #############
    with torch.no_grad():
        model.eval()
        for images, labels in valid_loader:
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()
                
            output = model(images)
            loss = criterion(output, labels)
            valid_loss += loss.item() * images.size(0)
            
    # Calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # Save model when validation loss decreases
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model ...'.format(valid_loss_min,valid_loss))
        
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.legend(frameon=False)
plt.xticks(range(20))
plt.show()

### Load the Model with Lowest Validation Loss

In [ ]:
model.load_state_dict(torch.load('model_cifar.pt'))

### Test the Trained Network

In [ ]:
y_pred = torch.empty((0,10))
y_true = torch.empty((0,1))

test_loss = 0.0

with torch.no_grad():
    model.eval()
    
    for images, labels in test_loader:
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
            
        output = model(images)
        loss = criterion(output, labels)
        test_loss += loss.item() * images.size(0)

        y_pred = torch.vstack([y_pred, output.cpu()])
        y_true = torch.vstack([y_true, labels.view(-1,1).cpu()])
        
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

In [ ]:
_, pred = torch.max(y_pred, 1)

for i in range(10):
    class_accuracies = multiclass_accuracy(pred, torch.squeeze(y_true).to(torch.int64), average=None, num_classes=10)
    
    print('Test Accuracy of %10s: %.2f%%' % (classes[i], 100 * class_accuracies[i].item()))

overall_accuracy = multiclass_accuracy(pred, torch.squeeze(y_true).to(torch.int64))    
print('\nTest Accuracy (Overall): %.2f%% ' % (100. * overall_accuracy.item()))

In [ ]:
# Plot the confusion matrix
disp = ConfusionMatrixDisplay(multiclass_confusion_matrix(pred, torch.squeeze(y_true).to(torch.int64), num_classes=10).numpy(), 
                              display_labels=classes)
disp.plot()
plt.xticks(rotation=90)
plt.show()

In [ ]:
test_precision = multiclass_precision(pred, torch.squeeze(y_true).to(torch.int64), num_classes=10)
test_recall =  multiclass_recall(pred, torch.squeeze(y_true).to(torch.int64), num_classes=10)

print('\nTest Precision : %.5f' % (test_precision))
print('\nTest Recall : %.5f' % (test_recall))

### Helper Function to Train with Multiple Learning Rates

In [ ]:
def train_model(learning_rate):
    # Create a new model
    model = Net()
    
    if train_on_gpu:
        model.cuda()
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training
    n_epochs = 20

    valid_loss_min = np.Inf

    train_losses, valid_losses = [], []

    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0

        ############# Train the model #############
        model.train()
        for images, labels in train_loader:
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()

            optimizer.zero_grad()

            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        ############# Train the model #############
        with torch.no_grad():
            model.eval()
            for images, labels in valid_loader:
                if train_on_gpu:
                    images, labels = images.cuda(), labels.cuda()

                output = model(images)
                loss = criterion(output, labels)
                valid_loss += loss.item() * images.size(0)

        # Calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
            
    return train_losses, valid_losses

In [ ]:
def plot_losses(train_losses, valid_losses, lr):
    plt.plot(train_losses, label='Training loss')
    plt.plot(valid_losses, label='Validation loss')
    plt.legend(frameon=False)
    plt.xticks(range(20))
    plt.title("Learning Rate: " + str(lr))
    plt.show()

In [ ]:
train_losses_0001, valid_losses_00001 = train_model(0.0001)

In [ ]:
plot_losses(train_losses_0001, valid_losses_00001, 0.0001)

In [ ]:
train_losses_01, valid_losses_01 = train_model(0.01)

In [ ]:
plot_losses(train_losses_01, valid_losses_01, 0.01)

In [ ]:
train_losses_1, valid_losses_1 = train_model(0.1)

In [ ]:
plot_losses(train_losses_1, valid_losses_1, 0.01)

In [ ]:
# Enlarge the graph
plt.plot(train_losses_1, label='Training loss')
plt.plot(valid_losses_1, label='Validation loss')
plt.legend(frameon=False)
plt.xticks(range(20))
plt.ylim((2.3,2.324))
plt.title("Learning Rate: " + str(0.1))
plt.show()